In [ ]:
!pip install numpy
!pip install tensorflow
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import Recall, Accuracy
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling1D, Input, Lambda
from tensorflow.keras.models import Model, load_model
import tensorflow as tf

# Definir la métrica personalizada para calcular el F1 Score
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)
        y_true = tf.argmax(y_true, axis=1)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Ruta a los datos de entrenamiento y validación
train_data_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/platos_de_comida/train'
validation_data_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/platos_de_comida/validation'
model_weights_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/Modelo/food_classification_transformer_weights.h5'

# Preprocesamiento de datos y aumentación de imágenes
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.1
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Cargar el extractor de características y el modelo ViT
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
num_classes = train_generator.num_classes

def create_model(num_classes):
    base_model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=num_classes, ignore_mismatched_sizes=True)

    # Crear un modelo Keras desde el modelo ViT de transformers
    inputs = Input(shape=(224, 224, 3))
    # Reordenar las dimensiones de la entrada
    reshaped_inputs = Lambda(lambda x: tf.transpose(x, [0, 3, 1, 2]))(inputs)

    # Pasar las entradas reordenadas a través del modelo ViT
    vit_outputs = base_model.vit(
        reshaped_inputs,
        training=False
    ).last_hidden_state

    x = GlobalAveragePooling1D()(vit_outputs)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Crear el modelo final
    model = Model(inputs=inputs, outputs=outputs)

    # Compilar el modelo
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', Recall(), F1Score()])

    return model

if os.path.exists(model_weights_path):
    print("Cargando modelo existente...")
    model = create_model(num_classes)
    model.load_weights(model_weights_path)
else:
    print("Creando un modelo nuevo...")
    model = create_model(num_classes)

# Configurar el callback para guardar el mejor modelo basado en la métrica de validación de precisión (val_accuracy)
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1, save_weights_only=True)

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# Cargar el mejor modelo guardado
model.load_weights(model_weights_path)

# Evaluar el modelo final en los datos de validación
loss, accuracy, recall, f1 = model.evaluate(validation_generator)
print(f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, F1 Score: {f1}")

# Guardar el modelo entrenado
model.save_weights(model_weights_path)
print("Modelo almacenado en:", model_weights_path)

Found 2527 images belonging to 23 classes.
Found 803 images belonging to 23 classes.
Cargando modelo existente...


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from google.colab import drive

# Montar Google Drive
#drive.mount('/content/drive')

# Ruta al modelo entrenado y a la carpeta de predicciones
model_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/Modelo/food_classification_ivg13.h5'
prediction_folder ='/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/prediccion'

# Verifique si el prediction_folder existe
if not os.path.exists(prediction_folder):
    os.makedirs(prediction_folder)

# Cargar el modelo entrenado
model = load_model(model_path)

# Diccionario de clases ajustado según tus clases
class_indices = {
    'Aguadito de pollo': 0, 'Aji de pollo y arroz': 1, 'Anticucho': 2, 'Arroz chaufa de pollo': 3,
    'Arroz con leche': 4, 'Arroz con pollo': 5, 'Causa rellena': 6, 'Cebiche de pescado': 7,
    'Chanfainita': 8, 'Choclo con queso': 9, 'Lomo saltado con arroz': 10, 'Mazamorra morada': 11,
    'Pan con chicharron': 12, 'Pan con pollo': 13, 'Papa a la huancaína': 14, 'Pollo a la brasa con papas fritas': 15,
    'Pollo broaster con papas fritas': 16, 'Sopa seca con carapulcra': 17, 'Tacacho con cecina': 18,
    'Tallarines rojos con pollo': 19, 'Tallarines saltados con pollo': 20, 'Tallarines verdes con bistec': 21,
    'Tamal de pollo': 22
}
class_labels = {v: k for k, v in class_indices.items()}  # Invertir el diccionario

# Función para preprocesar la imagen
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalizar la imagen
    return image

# Leer y predecir cada imagen en la carpeta de predicciones
for file_name in os.listdir(prediction_folder):
    image_path = os.path.join(prediction_folder, file_name)
    image = preprocess_image(image_path)

    # Realizar la predicción
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    confidence = np.max(prediction) * 100  # Calcular el porcentaje de confianza

    # Mostrar la imagen con la etiqueta predicha y el porcentaje de confianza
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f"Predicción: {predicted_label} - Confianza: {confidence:.2f}%")
    plt.axis('off')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.